In [59]:
import pandas as pd
import numpy as np

In [60]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 1

### Raw data

In [61]:
%%bigquery StartedApps
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    new_loan_type as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2

Query is running:   0%|          |

Downloading:   0%|          |

In [62]:
%%bigquery SubmittedApps
SELECT 
    DATE_TRUNC( termsAndConditionsSubmitDateTime,MONTH) as mm,  
    new_loan_type as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2

Query is running:   0%|          |

Downloading:   0%|          |

In [63]:
%%bigquery ApprovedApps
SELECT 
    DATE_TRUNC( decision_date,MONTH) as mm, 
    new_loan_type as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where flagApproval = 1

group by 1,2
order by 1,2

Query is running:   0%|          |

Downloading:   0%|          |

In [64]:
%%bigquery BookedApps
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    new_loan_type as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2

Query is running:   0%|          |

Downloading:   0%|          |

In [65]:
%%bigquery BookedAmt
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    new_loan_type as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2

Query is running:   0%|          |

Downloading:   0%|          |

In [66]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [68]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

### Loan Account Tagging

In [69]:
%%bigquery loan_type
  SELECT
    loanAccountNumber,
    new_loan_type as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1


Query is running:   0%|          |

Downloading:   0%|          |

In [71]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [73]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [74]:
%%bigquery vas_df
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
     new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1,2,3

Query is running:   0%|          |

Downloading:   0%|          |

In [75]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

### export

In [77]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)